In [2]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"



from keras.applications.vgg16 import VGG16
vggmodel = VGG16(include_top=False,input_shape=(128,128,3),weights="imagenet",classes=2)
newmodel = Sequential() 
for i, layer in enumerate(vggmodel.layers):
    if i<15:         
      newmodel.add(layer)
for layer in newmodel.layers:
  layer.trainable=False    

In [3]:
gender_128 = load_model('gender_model_128.model')
age_128 = load_model('age_model_128.model')
upscale_32 = load_model('high_resol1_128_1x.model')
upscale_128= load_model('high_resol2_128_1x.model')



In [7]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv
import matplotlib.pyplot as plt





gender_128 = load_model('gender_model_128.model')
age_128 = load_model('age_model_128.model')
upscale_32 = load_model('high_resol1_128_1x.model')
upscale_128= load_model('high_resol2_128_1x.model')


# open webcam
webcam = cv2.VideoCapture(0)
    
gender_classes = ['man','woman']
age_classes=["0 to 10","10 to 20","20 to 30","30 to 40","40 to 50","50 to 60","60 to 70","70 to 80","80 to 90","90 to 100","100 to 120"]

# loop through frames
while webcam.isOpened():

    # read frame from webcam 
    status, frame = webcam.read()
    print(frame.shape,type(frame))
    face, confidence = cv.detect_face(frame)


    # loop through detected faces
    for idx, f in enumerate(face):

        # get corner points of face rectangle        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)

        # crop the detected face region
        face_crop = np.copy(frame[startY:endY,startX:endX])

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue
        
        # preprocessing for gender detection model
        if(face_crop.shape[0]>80): 
            face_crop=cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
            face_crop = cv2.resize(face_crop, (128,128))
            face_crop = face_crop.astype("float") / 255.0
            face_crop = img_to_array(face_crop)
            face_crop = np.expand_dims(face_crop, axis=0)
            face_crop1 = face_crop
                
        else:
            face_crop=cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
            face_crop = cv2.resize(face_crop, (32,32))
            face_crop = face_crop.astype("float") / 255.0
            face_crop = img_to_array(face_crop)
            face_crop = np.expand_dims(face_crop, axis=0)
            face_crop1 = upscale_32.predict(face_crop)

        
#         pred21 = upscale_128.predict(face_crop)
        pred2 = upscale_128.predict(face_crop1)
        pred3 = newmodel.predict(pred2)
        
        
        age_result = age_128.predict(pred3)
        gender_result = gender_128.predict(pred3)
        
        
#         ximg =  np.clip(pred2,0.0,1.0).reshape(128, 128,3)
#         plt.figure(figsize=(15,15))
#         plt.subplot(1,2,1)
#         plt.imshow(face_crop[0])
#         plt.subplot(1,2,2) 
#         plt.imshow(ximg)
#         plt.show()
        
        
        # get label with max accuracy
        gender_idx = np.argmax(gender_result)
        gender = gender_classes[gender_idx]
        age_idx = np.argmax(age_result)
        age = age_classes[age_idx]
        
        label = f"gender: {gender}, age :{age}"
        Y = startY - 10 if startY - 10 > 10 else startY + 10

        # write label and confidence above face rectangle
        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    # display output
    cv2.imshow("gender detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()

(480, 640, 3) <class 'numpy.ndarray'>
1/1 [==============================] - 0s 59ms/step
(480, 640, 3) <class 'numpy.ndarray'>
1/1 [==============================] - 0s 15ms/step
(480, 640, 3) <class 'numpy.ndarray'>
1/1 [==============================] - 0s 14ms/step
(480, 640, 3) <class 'numpy.ndarray'>
1/1 [==============================] - 0s 16ms/step
(480, 640, 3) <class 'numpy.ndarray'>
1/1 [==============================] - 0s 15ms/step
(480, 640, 3) <class 'numpy.ndarray'>
1/1 [==============================] - 0s 14ms/step
(480, 640, 3) <class 'numpy.ndarray'>
1/1 [==============================] - 0s 14ms/step
(480, 640, 3) <class 'numpy.ndarray'>
1/1 [==============================] - 0s 14ms/step
